## 3D Isotropic Quantum Harmonic Oscillator - Dynamic Simulation
### Case 3.2 - Expansion of Cartesian and Spherical Eigenstates

<br>

Static simulation of the expansion of:
* cartesian eigenstate $\psi_{0,1,2}(x,y,z)$ into spherical eigenstates $\psi_{n,l}^{m}(r,\theta,\phi)$ ,
* spherical eigenstate $\psi_{0,2}^{-1}(r,\theta,\phi)$ into cartesian eigenstates $\psi_{n_x,n_y,n_z}(x,y,z)$ ,

that satisfy the condition $q = 2n + l = n_x + n_y + n_z$ .


#### Import libraries and functions


In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import qho_eigen as qho
from math_tools import *


#### Initialize cartesian eigenstate $\psi_{0,1,2}(x,y,z)$


In [2]:
# Define physical constants
hbar = 1
M = 2
w = 1
s = math.sqrt(2 * hbar / (M * w))

# Set up coordinate system
x_min = -4
x_max = 4
dx = 0.05
x = np.linspace(x_min, x_max, int((x_max - x_min) / dx) + 1) # np.linspace handles edges better

y_min = -4
y_max = 4
dy = 0.05
y = np.linspace(y_min, y_max, int((y_max - y_min) / dy) + 1) # np.linspace handles edges better

z_min = -4
z_max = 4
dz = 0.05
z = np.linspace(z_min, z_max, int((z_max - z_min) / dz) + 1) # np.linspace handles edges better

Y, Z, X = np.meshgrid(y, z, x) # This is because of NumPy's row-major order
R, THETA, PHI = cart2sph(X, Y, Z)

# Compute cartesian eigenstate
nx = 0
ny = 1
nz = 2
psi_cart = qho.eigen3D_cart(s, nx, ny, nz, X, Y, Z)


#### Expand into spherical eigenstates $\psi_{n,l}^{m}(r,\theta,\phi)$


In [3]:
# Compute spherical eigenstates
N_sph = 10
n = np.vstack(np.array([ 1,  1,  1,  0,  0,  0,  0,  0,  0,  0]), dtype=object)
l = np.vstack(np.array([ 1,  1,  1,  3,  3,  3,  3,  3,  3,  3]), dtype=object)
m = np.vstack(np.array([-1,  0,  1, -3, -2, -1,  0,  1,  2,  3]), dtype=object)
eigenfuns_sph = np.zeros((N_sph, np.size(z), np.size(y), np.size(x)), dtype=np.complex128)

for i in range(N_sph):
    eigenfuns_sph[i] = qho.eigen3D_sph(s, n[i][0], l[i][0], m[i][0], R, THETA, PHI)

# Compute expansion coefficients
c = np.vstack(np.zeros((N_sph, 1)), dtype=np.complex128)

for i in range(N_sph):
    c[i] = np.trapz(np.trapz(np.trapz(np.conjugate(eigenfuns_sph[i]) * psi_cart, x=x, axis=2), x=y, axis=1), x=z)

print('Expansion coefficients:')
for i in range(N_sph): print(f'n  {n[i][0]}, l  {l[i][0]}, m  {m[i][0]} -> {c[i][0]:.6f}')
print()

# Reconstruct cartesian state
psi_sph = np.sum(c[:, np.newaxis, np.newaxis] * eigenfuns_sph, axis=0)

# Compute absolute error of spherical expansion
print(f'Max abs error of spherical expansion = {np.max(np.abs(abs2(psi_cart) - abs2(psi_sph)))}')
print()


Expansion coefficients:
n  1, l  1, m  -1 -> -0.000000-0.316228j
n  1, l  1, m  0 -> -0.000000+0.000000j
n  1, l  1, m  1 -> 0.000000-0.316228j
n  0, l  3, m  -3 -> -0.000000+0.000000j
n  0, l  3, m  -2 -> 0.000000+0.000000j
n  0, l  3, m  -1 -> 0.000000+0.632456j
n  0, l  3, m  0 -> 0.000000+0.000000j
n  0, l  3, m  1 -> -0.000000+0.632456j
n  0, l  3, m  2 -> -0.000000-0.000000j
n  0, l  3, m  3 -> -0.000000+0.000000j

Max abs error of spherical expansion = 1.3916090502164025e-12



#### Initialize spherical eigenstate $\psi_{0,2}^{-1}(r,\theta,\phi)$


In [4]:
# Define physical constants
hbar = 1
M = 2
w = 1
s = math.sqrt(2 * hbar / (M * w))

# Set up coordinate system
r_min = 0
r_max = 6
r_N = 201
r = np.linspace(r_min, r_max, r_N)

theta_min = 0
theta_max = math.pi
theta_N = 201
theta = np.linspace(theta_min, theta_max, theta_N)

phi_min = 0
phi_max = 2 * math.pi
phi_N = 201
phi = np.linspace(phi_min, phi_max, phi_N)

THETA, PHI, R = np.meshgrid(theta, phi, r) # This is because of NumPy's row-major order
X, Y, Z = sph2cart(R, THETA, PHI)

# Compute spherical eigenstates
n = 0
l = 2
m = -1
psi_sph = qho.eigen3D_sph(s, n, l, m, R, THETA, PHI)


#### Expand into cartesian eigenstates $\psi_{n_x,n_y,n_z}(x,y,z)$


In [5]:
# Compute cartesian eigenstates
N_cart = 6
nx = np.vstack(np.array([1, 1, 0, 2, 0, 0]), dtype=object)
ny = np.vstack(np.array([1, 0, 1, 0, 2, 0]), dtype=object)
nz = np.vstack(np.array([0, 1, 1, 0, 0, 2]), dtype=object)

eigenfuns_cart = np.zeros((N_cart, np.size(phi), np.size(theta), np.size(r)))

for i in range(N_cart):
    eigenfuns_cart[i] = qho.eigen3D_cart(s, nx[i][0], ny[i][0], nz[i][0], X, Y, Z)

# Compute expansion coefficients
c = np.vstack(np.zeros((N_cart, 1)), dtype=np.complex128)

for i in range(N_cart):
    c[i] = np.trapz(np.trapz(np.trapz(np.conjugate(eigenfuns_cart[i]) * psi_sph * R**2 * np.sin(THETA), x=r, axis=2), x=theta, axis=1), x=phi)

print('Expansion coefficients:')
for i in range(N_cart): print(f'nx {nx[i][0]}, ny {ny[i][0]}, nz {nz[i][0]} -> {c[i][0]:.6f}')
print()

# Reconstruct spherical state
psi_cart = np.sum(c[:, np.newaxis, np.newaxis] * eigenfuns_cart, axis=0)

# Compute absolute error of cartesian expansion
print(f'Max abs error of cartesian expansion = {np.max(np.abs(abs2(psi_sph) - abs2(psi_cart)))}')
print()


Expansion coefficients:
nx 1, ny 1, nz 0 -> 0.000000+0.000000j
nx 1, ny 0, nz 1 -> 0.707107-0.000000j
nx 0, ny 1, nz 1 -> 0.000000-0.707107j
nx 2, ny 0, nz 0 -> -0.000000-0.000000j
nx 0, ny 2, nz 0 -> 0.000000-0.000000j
nx 0, ny 0, nz 2 -> 0.000000+0.000000j

Max abs error of cartesian expansion = 1.0460517452237639e-09

